In [28]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
sns.set_context("talk")
import re
import nltk

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [29]:
articles = pd.read_csv("use_riskEx.csv")
articles.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,"Cecchetti, Schoenholtz",Research-based policy analysis and commentary ...,"Despite recent technological advances, the cos...",Voxeu.org,https://voxeu.org/article/stubbornly-high-cost...,2018-03-27 00:00:00+00:00,The stubbornly high cost of remittances
1,Reuters,Sign up now to get free exclusive access to re...,Twitter Inc will start banning cryptocurrency ...,Cio.com.au,https://www.cio.com.au/article/635378/twitter-...,2018-03-26 23:43:50+00:00,Twitter to ban cryptocurrency ads
4,newsfeeds@nzherald.co.nz,\t NEW YORK (AP) — Twitter says it will ban or...,NEW YORK (AP) — Twitter says it will ban or re...,Nzherald.co.nz,http://www.nzherald.co.nz/business/news/articl...,2018-03-26 23:25:06+00:00,"Twitter to ban cryptocurrency ads, joining Fac..."
5,"David Cottle, Analyst, David Cottle",Get the best parts of DailyFX.com in the new D...,"The New Zealand Dollar had a busy morning, wit...",Dailyfx.com,https://www.dailyfx.com/forex/market_alert/201...,2018-03-26 23:19:00+00:00,"New Zealand Dollar Firm After Trade Beat,Eyes ..."
6,Kellie Ell,\n\n\n\n var postLoadFunctions = {}...,One expert said the majority of initial coin o...,CNBC,https://www.cnbc.com/2018/03/26/how-to-evaluat...,2018-03-26 23:18:00+00:00,How to evaluate risky initial coin offerings a...


In [30]:
articles = articles.reset_index() #Fixing index issues, this code is not needed if index is fine
articles = articles.drop(['index'], axis = 1)

In [31]:
articles['timeStamp'] = pd.to_datetime(articles['timeStamp'])
articles['timeStamp'] = articles['timeStamp'].dt.tz_localize('UTC')

In [32]:
articles = articles.drop_duplicates(subset = 'title')

In [65]:
#this code removes all punctuation and alphanumeric words from the content

eng_stopwords = nltk.corpus.stopwords.words('english')

removed_punct=[]
for text in articles["contents"]:
    text=re.sub('[^a-zA-Z ]' ,'',text)
    text = [word for word in text.split() if not word in eng_stopwords]
    text = (' '.join(text))
    removed_punct.append(text)

In [66]:
#performing Tf-idf
all_documents= list(removed_punct)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True)
tfidf_feat = tfidf.fit_transform(all_documents)

In [67]:
features = pd.DataFrame(tfidf_feat.toarray())

In [80]:
###########
##K-Means##
###########

#cluster_count = 15
#model = KMeans(n_clusters = cluster_count, init='k-means++', max_iter=100, n_init=1)

model = KMeans(init='k-means++', max_iter=100, n_init=1)
model.fit(features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=8, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [81]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf.get_feature_names()
for i in range(cluster_count):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 erc
 coinbase
 assets
 support
 standard
 gdax
 post
 transactions
 exchange
 token
Cluster 1:
 ads
 twitter
 google
 banning
 facebook
 joins
 ban
 cryptocurrency
 etbitcoin
 hovering
Cluster 2:
 percent
 loan
 students
 march
 cointelegraph
 student
 franchise
 college
 please
 trading
Cluster 3:
 instanceof
 var
 slotid
 function
 rtvar
 admantxurl
 adunit
 mpsgetad
 rfunction
 mpsext
Cluster 4:
 city
 atlanta
 ransomware
 attack
 bottoms
 computer
 citys
 officials
 pay
 employees
Cluster 5:
 cboe
 futures
 markets
 jan
 marketwatch
 letter
 april
 mar
 range
 traded
Cluster 6:
 twitter
 ads
 ban
 securityuknws
 advertising
 offerings
 policy
 exchanges
 google
 cryptocurrency
Cluster 7:
 one
 like
 blockchain
 the
 people
 bitcoin
 gold
 back
 would
 time
Cluster 8:


IndexError: index 8 is out of bounds for axis 0 with size 8

In [75]:
model.labels_

array([ 1,  7,  2,  4, 13,  1,  3, 10,  6,  8,  7,  3, 12,  1, 14,  4, 14,
        9, 11,  9,  1,  3,  0,  0,  3,  1,  1,  0, 10,  1,  1,  1,  3,  0,
        4,  6,  4,  2, 13,  3,  2,  1,  1,  1,  1, 14,  1,  9, 10, 14,  1,
        1, 10,  6,  1,  5,  2,  8,  5,  8, 10,  1,  2,  8,  6,  5,  6,  8,
        0, 11, 11,  1, 12, 14,  8,  2], dtype=int32)

In [82]:
articles['cluster'] = model.labels_
articles.head()

,author,contents,description,publisher,source_url,timeStamp,title,cluster
0,"Cecchetti, Schoenholtz",Research-based policy analysis and commentary ...,"Despite recent technological advances, the cos...",Voxeu.org,https://voxeu.org/article/stubbornly-high-cost...,2018-03-27 00:00:00+00:00,The stubbornly high cost of remittances,0
1,Reuters,Sign up now to get free exclusive access to re...,Twitter Inc will start banning cryptocurrency ...,Cio.com.au,https://www.cio.com.au/article/635378/twitter-...,2018-03-26 23:43:50+00:00,Twitter to ban cryptocurrency ads,6
2,newsfeeds@nzherald.co.nz,\t NEW YORK (AP) — Twitter says it will ban or...,NEW YORK (AP) — Twitter says it will ban or re...,Nzherald.co.nz,http://www.nzherald.co.nz/business/news/articl...,2018-03-26 23:25:06+00:00,"Twitter to ban cryptocurrency ads, joining Fac...",6
3,"David Cottle, Analyst, David Cottle",Get the best parts of DailyFX.com in the new D...,"The New Zealand Dollar had a busy morning, wit...",Dailyfx.com,https://www.dailyfx.com/forex/market_alert/201...,2018-03-26 23:19:00+00:00,"New Zealand Dollar Firm After Trade Beat,Eyes ...",0
4,Kellie Ell,\n\n\n\n var postLoadFunctions = {}...,One expert said the majority of initial coin o...,CNBC,https://www.cnbc.com/2018/03/26/how-to-evaluat...,2018-03-26 23:18:00+00:00,How to evaluate risky initial coin offerings a...,3


In [83]:
articles.sort_values(by = 'cluster')

,author,contents,description,publisher,source_url,timeStamp,title,cluster
0,"Cecchetti, Schoenholtz",Research-based policy analysis and commentary ...,"Despite recent technological advances, the cos...",Voxeu.org,https://voxeu.org/article/stubbornly-high-cost...,2018-03-27 00:00:00+00:00,The stubbornly high cost of remittances,0
21,Adam James,"Real time prices""vires in numeris.""\tBitcoin\t...",Coinbase has announced its intention to suppor...,Bitcoinist.com,http://bitcoinist.com/coinbase-announces-plans...,2018-03-26 21:54:49+00:00,Coinbase Announces Plans for ERC20 Support,0
26,Deutsche Welle (www.dw.com),Police in southeastern Spain have arrested the...,Police in southeastern Spain have arrested the...,Dw.com,http://www.dw.com/en/spanish-police-nab-allege...,2018-03-26 21:38:00+00:00,Spanish police nab alleged 'mastermind' of ban...,0
38,Scott Scanlon,Now they could be staring down some major tax ...,Now they could be staring down some major tax ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/4-th...,2018-03-26 21:03:18+00:00,4 things to know about your cryptocurrency at ...,0
50,Camila Russo,Connecting decision makers to a dynamic networ...,"Coinbase Inc., which owns one of the largest U...",Bloomberg,https://www.bloomberg.com/news/articles/2018-0...,2018-03-26 20:21:15+00:00,Crypto Exchange Coinbase Adds Support for Ethe...,0
58,Scott Scanlon,Multiple security firms recently identified cr...,Multiple security firms recently identified cr...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/who-...,2018-03-26 20:03:27+00:00,Who and What Is Coinhive?,0
20,Guest Post,The Japanese government has made major strides...,The Japanese government has made major strides...,Valuewalk.com,https://www.valuewalk.com/2018/03/virtual-exch...,2018-03-26 22:11:23+00:00,Crackdown On Virtual Exchanges: Will They Brin...,0
19,Colin Harper,Ready to start building Dapps? Dive deep into ...,One of the world’s most popular cryptocurrency...,Bitcoinmagazine.com,https://bitcoinmagazine.com/articles/coinbase-...,2018-03-26 22:11:25+00:00,Coinbase to Support ERC20 Technical Standards ...,0
5,"Amanda Lee, Josh Ye, Amanda Lee, Josh Ye","\tBlockchain\t‘Where there’s demand, there wil...",Initial coin offerings (ICOs) may well have be...,Scmp.com,http://www.scmp.com/tech/china-tech/article/21...,2018-03-26 23:15:26+00:00,How agents are helping investors buy into init...,0
75,Laurel Loomis Rimon,Ready to start building Dapps? Dive deep into ...,Compliance with Anti-Money Laundering (AML) ru...,Bitcoinmagazine.com,https://bitcoinmagazine.com/articles/op-ed-qui...,2018-03-26 18:53:35+00:00,Op Ed: A Quick-Start Token Sale Compliance Gui...,0
